In [10]:
import sys
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)

egresados=pd.read_sql("""select egresados.*, codigos.description from egresados inner join codigos on egresados.status=codigos.code    where muestra = 3 """,cnx)
print('len de egresados',len(egresados))
correos=pd.read_sql('select * from correos',cnx)
carreras=pd.read_sql('select * from carreras',cnx)
egresados=egresados.loc[~egresados['status'].isin([1,2,10])]
writer = pd.ExcelWriter('storage/correos_muestra_sin_contestar.xlsx', engine='xlsxwriter')

workbook = writer.book
a_color='#173d83'
#estilos----------------
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13}) 
header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':12})
blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color})
date_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':10,
    'border_color':a_color,
    'num_format': 'dd/mm/yy'})
date_content_bold = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12,
    'bold': True,
    'num_format': 'dd/mm/yy'})
worksheet = workbook.add_worksheet()
worksheet.merge_range('C2:H3', 'PROGRAMA DE VINCULACION A EGRESADOS UNAM', negro_b)
worksheet.merge_range('C4:H4', 'EGRESADOS QUE NO HAN CONTESTADO 2020', negro_b)
worksheet.insert_image("A1", "img/logoPVE.png",{"x_scale": 0.2, "y_scale": 0.2})
worksheet.merge_range('G6:H6',today, date_content_bold)

worksheet.write('B8','Estatus',header_format)
worksheet.write('C8','Fecha en que se marco',header_format)
worksheet.write('D8','Nombre',header_format)
worksheet.write('E8','Paterno',header_format)
worksheet.write('F8','Materno',header_format)
worksheet.write('G8','Numero de cuenta',header_format)
worksheet.write('H8','Carrera',header_format)
worksheet.write('I8','Plantel',header_format)
worksheet.write('J8','Correo 1',header_format)
worksheet.write('K8','Correo 2',header_format)
worksheet.write('L8','Correo 3',header_format)
worksheet.write('M8','Correo 4',header_format)

for i in range(0,len(egresados)):
    
    correos_eg=correos.loc[correos['cuenta']==egresados['cuenta'].values[i]]
    # print(egresados['cuenta'].values[i],len(correos_eg))
    worksheet.write('B'+str(i+9),egresados['description'].values[i],blue_content)
    worksheet.write('C'+str(i+9),str(egresados['updated_at'].values[i])[0:10],blue_content)
    worksheet.write('D'+str(i+9),egresados['nombre'].values[i],blue_content)
    worksheet.write('E'+str(i+9),egresados['paterno'].values[i],blue_content)
    worksheet.write('F'+str(i+9),egresados['materno'].values[i],blue_content)
    worksheet.write('G'+str(i+9),egresados['cuenta'].values[i],blue_content)
    worksheet.write('H'+str(i+9),carreras.loc[carreras['clave_carrera']==egresados['carrera'].values[i],'carrera'].values[0],blue_content)
    worksheet.write('I'+str(i+9),carreras.loc[carreras['clave_plantel']==egresados['plantel'].values[i],'plantel'].values[0],blue_content)
    for j in range(len(correos_eg)):
        worksheet.write(i+8,9+j,correos_eg['correo'].values[j],blue_content)

worksheet.set_column('B:D',17)
worksheet.set_column('E:E',20)
worksheet.set_column('F:M',28)
worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


len de egresados 19201


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,cuenta,nombre,paterno,materno,carrera,plantel,extras,promedio,correo,...,bach,correo_unam,sistema,muestra,status,llamadas,updated_at,sexo,fec_nac,actualized
0,107649,308198625,SILVIA,MIGUEL,ZAMBRANO,302,200,12,8.55,silviamz92@hotmail.com,...,33.0,silviamzambrano@comunidad.unam.mx,None,3,0,0,NaT,None,None,NaT
1,110829,311201794,JESSICA SHEILA,TORRES,ALVAREZ,208,300,1,8.07,jessy_torr95@hotmail.com,...,31.0,jess.ta@comunidad.unam.mx,None,3,11,0,2024-02-06 15:35:29,None,None,NaT
3,147427,89369177,JUAN,RODRIGUEZ,MORALES,102,1,11,833.00,0,...,27.0,None,E,3,0,0,NaT,M,1972-08-26,NaT
4,147429,89399097,LUIS,LARA,RAZO,102,1,10,764.00,0,...,29.0,None,E,3,0,0,NaT,M,1973-03-24,NaT
5,147432,91197683,MARIBEL,MORALES,FERNANDEZ,102,1,4,814.00,0,...,31.0,None,E,3,0,0,NaT,F,1975-12-28,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,175154,123123444,NOMBRE DE,PRUEBA,NO4,102,1,4,7.40,correo_De_prueba@hotmail.com,...,22.0,None,E,3,10,20,2024-04-08 20:08:03,F,0998-08-07,NaT
19197,175155,123123555,NOMBRE DE,PRUEBA,NO5,102,1,4,7.40,correo_De_prueba@hotmail.com,...,22.0,None,E,3,10,0,2024-04-23 17:13:28,M,0998-08-07,NaT
19198,175156,123123666,NOMBRE DE,PRUEBA,NO6,102,1,4,7.40,correo_De_prueba@hotmail.com,...,22.0,None,E,3,10,5,2024-04-11 15:56:42,F,0998-08-07,NaT
19199,175157,123123777,NOMBRE DE,PRUEBA,NO7,102,1,4,7.40,correo_De_prueba@hotmail.com,...,22.0,None,E,3,10,0,2024-04-23 17:33:27,M,0998-08-07,NaT
